In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoSuchWindowException
import time
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC

In [3]:
body_type = ["hatchback", 'sedan', 'suv', 'luxury%20sedan', 'luxury%20suv'] 
city_id = ["777", "1692","769" , "6", "648", "1605"]
city_pin = ["700001", "380001","134101", "121001", "140604", "394101"]

In [4]:
# city link genration
city_links = []
for i in range(len(city_id)):
    for bodyType in body_type:
        link = "https://www.cars24.com/buy-used-car?bodyType="+bodyType+"&sort=P&storeCityId="+city_id[i]+"&pinId="+city_pin[i]
        print(link)
        city_links.append(link)

https://www.cars24.com/buy-used-car?bodyType=hatchback&sort=P&storeCityId=777&pinId=700001
https://www.cars24.com/buy-used-car?bodyType=sedan&sort=P&storeCityId=777&pinId=700001
https://www.cars24.com/buy-used-car?bodyType=suv&sort=P&storeCityId=777&pinId=700001
https://www.cars24.com/buy-used-car?bodyType=luxury%20sedan&sort=P&storeCityId=777&pinId=700001
https://www.cars24.com/buy-used-car?bodyType=luxury%20suv&sort=P&storeCityId=777&pinId=700001
https://www.cars24.com/buy-used-car?bodyType=hatchback&sort=P&storeCityId=1692&pinId=380001
https://www.cars24.com/buy-used-car?bodyType=sedan&sort=P&storeCityId=1692&pinId=380001
https://www.cars24.com/buy-used-car?bodyType=suv&sort=P&storeCityId=1692&pinId=380001
https://www.cars24.com/buy-used-car?bodyType=luxury%20sedan&sort=P&storeCityId=1692&pinId=380001
https://www.cars24.com/buy-used-car?bodyType=luxury%20suv&sort=P&storeCityId=1692&pinId=380001
https://www.cars24.com/buy-used-car?bodyType=hatchback&sort=P&storeCityId=769&pinId=13410

In [5]:
driver = webdriver.Chrome('chromedriver')
#driver.get(hatchback)

In [6]:
# web scraping links for each car for each city and each body type
links = []
for city_link in city_links:
    driver.get(city_link)
    time.sleep(3)
    for i in range(3):
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)") #scrolling down the page
        time.sleep(3)
    page = driver.find_elements_by_xpath("//div[@class = 'col-4']/a")
    car_links = [page.get_attribute('href') for page in page]
    links = links + car_links

In [301]:
# saving list of links as text file for future use
all_links=open('df_set2.txt', 'w')
for i in links:
    all_links.write(i+"\n")
all_links.close()

In [4]:
driver = webdriver.Chrome("chromedriver")

In [6]:
#list of the features being collected for each car
feat_list = ['Roof', 'Bonnet/Hood', 'Dicky Door/Boot Door', 'Boot Floor', 'Quarter Panel\nLHS\nRHS', 'Fender\nLHS\nRHS', 
             'Pillars\nLHS B\nLHS A\nLHS C\nRHS A\nRHS B\nRHS C', 'Apron\nLHS\nRHS', 'Firewall', 'Cowl Top', 'Bonnet Patti', 
             'Lower Cross Member', 'Radiator Support', 'Head Light Support', 'Running Border\nLHS\nRHS', 
             'Doors\nLHS Front\nLHS Rear\nRHS Rear\nRHS Front', 'Windshield\nFront\nRear', 'Pillars\nLHS A\nLHS B\nLHS C\nRHS A\nRHS B\nRHS C',
             'Lights\nLHS Taillight\nRHS Taillight\nLHS Headlight\nRHS Headlight', 'Bumpers\nFront\nRear', 
             'ORVM- Manual/Electric\nLHS\nRHS', 'Lights\nLHS Fog Light\nLHS Headlight\nLHS Taillight\nRHS Fog Light\nRHS Headlight\nRHS Taillight',
             'Tyre\nLHS Front Tyre\nRHS Front Tyre\nLHS Rear Tyre\nRHS Rear Tyre\nSpare Tyre', 
             'Engine', 'Engine Sound', 'Exhaust Smoke', 'Engine Blow By Back Compression', 'Engine Mounting', 
             'Clutch', 'Gear Shifting', 'Engine Oil', 'Battery', 'Coolant', 'AC Cooling', 'Heater', 
             'Music System', 'Leather Seats', 'Steering', 'Suspension', 'Brakes']
# feature dictionary
feat_dictionary = dict.fromkeys(feat_list, 1)
keys = list(feat_dictionary.keys())

In [8]:
    
def scrap(link):
    driver.get(link)
    time.sleep(5)
    feat_dictionary = dict.fromkeys(feat_list, 0)
    feat_dictionary["car_id"] = link.split("-")[-1].replace("/","")
    feat_dictionary["city"] = link.split("-")[-2].replace("/","")
    data = driver.find_elements_by_xpath("//li[@class = 'tHlIu']")
    data = [d.text.split("\n") for d in data]
    for d in data:
        if d[0]=="Insurance":
            feat_dictionary["insurance_type"] = d[-1]
            feat_dictionary["insurance_valid_m"] = d[-2].split()[-2]
            feat_dictionary["insurance_valid_y"] = d[-2].split()[-1]
        if d[0]=="History":
            feat_dictionary["history"] = d[-1]
        if d[0]=="Owner":
            feat_dictionary["owner"] = int(d[-1][0])
        if d[0]=="Kilometers Driven":
            feat_dictionary["kilometers_driven"] = int(d[-1].replace("km","").replace(",",""))
        if d[0] == "Fuel Type":
            feat_dictionary["Fuel Type"] = d[-1]
        if d[0] == "Transmission":
            feat_dictionary["Transmission"]=(d[-1])
        if d[0] == "Year of Purchase":
            feat_dictionary["Year of Purchase"] = (d[-1].split()[-1])
        price = int(driver.find_element_by_xpath("//li[@class = '_2DkPb']/span[2]").text.replace(",","").replace("₹","").strip())
        feat_dictionary["price"] = price
    time.sleep(3)
    data2 = driver.find_elements_by_xpath("//div[@class = '_24NRL']")
    data2_= [d.text for d in data2]
    for key in keys:
        for i in data2_:
            if key in i:
                feat_dictionary[key] = 1
                if len(key)<len(i):
                    i = i.replace(key,"").strip()
                    if i in ["Sound", "Blow By Back Compression", "Mounting", "Oil","Mounted Audio Controls"]:
                        pass
                    else:
                        feat_dictionary[key] = 0
    return feat_dictionary

In [10]:
# sample testing of function defined
scrap("https://www.cars24.com/buy-used-Hyundai-Elite-i20-2015-cars-Kolkata-1194483303/")

{'Roof': 1,
 'Bonnet/Hood': 1,
 'Dicky Door/Boot Door': 1,
 'Boot Floor': 1,
 'Quarter Panel\nLHS\nRHS': 0,
 'Fender\nLHS\nRHS': 1,
 'Pillars\nLHS B\nLHS A\nLHS C\nRHS A\nRHS B\nRHS C': 0,
 'Apron\nLHS\nRHS': 1,
 'Firewall': 1,
 'Cowl Top': 1,
 'Bonnet Patti': 1,
 'Lower Cross Member': 1,
 'Radiator Support': 1,
 'Head Light Support': 1,
 'Running Border\nLHS\nRHS': 1,
 'Doors\nLHS Front\nLHS Rear\nRHS Rear\nRHS Front': 0,
 'Windshield\nFront\nRear': 1,
 'Pillars\nLHS A\nLHS B\nLHS C\nRHS A\nRHS B\nRHS C': 0,
 'Lights\nLHS Taillight\nRHS Taillight\nLHS Headlight\nRHS Headlight': 0,
 'Bumpers\nFront\nRear': 1,
 'ORVM- Manual/Electric\nLHS\nRHS': 1,
 'Lights\nLHS Fog Light\nLHS Headlight\nLHS Taillight\nRHS Fog Light\nRHS Headlight\nRHS Taillight': 0,
 'Tyre\nLHS Front Tyre\nRHS Front Tyre\nLHS Rear Tyre\nRHS Rear Tyre\nSpare Tyre': 1,
 'Engine': 1,
 'Engine Sound': 1,
 'Exhaust Smoke': 1,
 'Engine Blow By Back Compression': 1,
 'Engine Mounting': 1,
 'Clutch': 1,
 'Gear Shifting': 1,
 '

In [24]:
import pandas as pd

In [ ]:
#driver = webdriver.Chrome("chromedriver")
final_list = []
for link in sample_link:
    time.sleep(3)
    try:
        final_list.append(scrap(link))
    except NoSuchWindowException:
        driver = webdriver.Chrome("chromedriver")
        time.sleep(3)
        final_list.append(scrap(link))

In [ ]:
# creating scraped data as dataframe
df = pd.DataFrame(final_list)
#df.to_csv("df_set8.csv")